# Classificação faixa TRL -  GPT-2 + Modelos ML


- https://huggingface.co/pierreguillou/gpt2-small-portuguese

## Código Versão 1.0 - 26 OUT 23

- Uso dos classificadores: *Modelos de ML*
- Matriz de confusão;
- Semente 42

- Esse código adota a simplificação proposta na dissertação, no sentido de obtermos a RV, depois alizarmos o k-fold. Dessa forma, temos uma econnomia computacional.

In [1]:
# dataset.csv   ou  dataset_pre_processado_1.csv  ou  dataset_pre_processado_stem_2.csv
#     CSV1                  CSV2                                   CSV3
dataset = "dataset.csv"

In [2]:
print("Lembre-se estamos usando o dataset: " + dataset)

Lembre-se estamos usando o dataset: dataset.csv


In [3]:
melhor_modelo = 'best_model_TRL_gpt2_ml_' + '.bin'
melhor_modelo

'best_model_TRL_gpt2_ml_.bin'

- Quantidade de tokens máxima = 242

In [4]:
MAX_LEN = 242

## Preparação

### Bibliotecas e ambiente

In [5]:
!pip install -qq transformers

In [6]:
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00


In [7]:
!nvidia-smi

Tue Dec 12 00:22:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
from torch import nn
from torch.optim import Adam
from transformers import GPT2Model, GPT2Tokenizer

In [9]:
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from collections import defaultdict
import textwrap

from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes, svm
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler

In [11]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy       : 1.23.5
pandas      : 1.5.3
torch       : 2.1.0+cu118
transformers: 4.35.2



# Particoes KFOLD

- Para mais detalhes, consulte o código *1-kfold.ipynb* em que as partições foram sorteadas. As células a seguir recriam o resultado obtido.

In [12]:
array1 = np.array([  0,   1,   2,   3,   5,   7,   9,  10,  11,  12,  13,  14,  15,
         16,  17,  19,  20,  21,  22,  23,  25,  26,  27,  28,  30,  32,
         33,  34,  35,  36,  37,  39,  40,  44,  45,  46,  47,  48,  49,
         50,  52,  53,  54,  55,  56,  57,  59,  60,  61,  63,  64,  66,
         68,  69,  70,  71,  72,  75,  76,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  87,  91,  92,  93,  94,  96,  97,  98,  99, 100,
        102, 103, 104, 105, 106, 108, 109, 110, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130,
        131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145,
        146, 148, 150, 151, 153, 154, 155, 157, 158, 159, 160, 161, 163,
        164, 165, 166, 167])

array2 = np.array([  0,   1,   3,   4,   5,   6,   7,   8,  10,  13,  14,  15,  16,
         17,  18,  19,  20,  22,  23,  24,  25,  26,  27,  29,  30,  31,
         34,  36,  37,  38,  40,  41,  42,  43,  44,  45,  47,  49,  51,
         52,  53,  54,  55,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  73,  74,  75,  76,  77,  80,  83,
         84,  86,  88,  89,  90,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102, 104, 105, 107, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 132, 133, 135, 136, 137, 138, 139, 142, 143, 144, 145,
        147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 161, 162,
        163, 165, 166, 167])
array3 = np.array([  1,   2,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         17,  18,  20,  21,  22,  23,  24,  25,  28,  29,  30,  31,  32,
         33,  35,  36,  37,  38,  39,  41,  42,  43,  44,  45,  46,  47,
         48,  50,  51,  52,  54,  55,  56,  57,  58,  59,  62,  63,  65,
         67,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
         82,  84,  85,  86,  87,  88,  89,  90,  91,  93,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111,
        112, 113, 114, 118, 120, 121, 122, 123, 124, 126, 128, 129, 131,
        132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 144, 146, 147,
        148, 149, 150, 151, 152, 153, 155, 156, 158, 159, 160, 161, 162,
        163, 164, 165, 166])
array4 = np.array([  0,   2,   3,   4,   6,   8,   9,  10,  11,  12,  15,  16,  18,
         19,  20,  21,  22,  24,  25,  26,  27,  28,  29,  31,  32,  33,
         34,  35,  37,  38,  39,  40,  41,  42,  43,  45,  46,  47,  48,
         49,  50,  51,  53,  56,  57,  58,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  71,  72,  73,  74,  75,  76,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  98, 100, 101, 103, 104, 106, 107, 108, 109, 110, 111,
        112, 114, 115, 116, 117, 119, 121, 123, 124, 125, 127, 128, 129,
        130, 131, 133, 134, 135, 137, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159,
        162, 163, 164, 166, 167])
array5 = np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  21,  23,  24,  26,  27,  28,  29,
         30,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,  43,
         44,  46,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,
         60,  61,  62,  64,  65,  66,  67,  68,  69,  70,  72,  73,  74,
         77,  78,  79,  81,  82,  83,  85,  86,  87,  88,  89,  90,  91,
         92,  94,  95,  96,  97,  99, 101, 102, 103, 105, 106, 107, 108,
        110, 111, 113, 115, 116, 117, 118, 119, 120, 122, 125, 126, 127,
        128, 130, 131, 132, 133, 134, 136, 137, 138, 140, 141, 143, 144,
        145, 146, 147, 148, 149, 151, 152, 154, 156, 157, 158, 159, 160,
        161, 162, 164, 165, 167])
# Criar a lista composta
kfold_train = [array1, array2, array3, array4, array5]

In [13]:
# Criar as cinco arrays numpy
array1 = np.array([  4,   6,   8,  18,  24,  29,  31,  38,  41,  42,  43,  51,  58, 62,  65,  67,  73,  74,  86,  88,  89,  90,  95, 101, 107, 111, 128, 133, 144, 147, 149, 152, 156, 162])
array2 = np.array([  2,   9,  11,  12,  21,  28,  32,  33,  35,  39,  46,  48,  50, 56,  72,  78,  79,  81,  82,  85,  87,  91, 103, 106, 108, 131, 134, 140, 141, 146, 148, 158, 159, 164])
array3 = np.array([  0,   3,  15,  16,  19,  26,  27,  34,  40,  49,  53,  60,  61, 64,  66,  68,  69,  83,  92,  94, 110, 115, 116, 117, 119, 125, 127, 130, 137, 143, 145, 154, 157, 167])
array4 = np.array([  1,   5,   7,  13,  14,  17,  23,  30,  36,  44,  52,  54,  55, 59,  70,  77,  96,  97,  99, 102, 105, 113, 118, 120, 122, 126, 132, 136, 138, 151, 160, 161, 165])
array5 = np.array([ 10,  20,  22,  25,  37,  45,  47,  57,  63,  71,  75,  76,  80,  84,  93,  98, 100, 104, 109, 112, 114, 121, 123, 124, 129, 135, 139, 142, 150, 153, 155, 163, 166])


# Criar a lista composta
kfold_test = [array1, array2, array3, array4, array5]

# Dataset

In [14]:
df = pd.read_csv(dataset)

In [15]:
novo_df = df[["resumo", "rotulo"]]

In [16]:
def adapto_faixa(faixa):
    faixa -=1
    return faixa

In [17]:
novo_df['rotulo'] = novo_df.rotulo.apply(adapto_faixa)

<ipython-input-17-d13f2641b9c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df['rotulo'] = novo_df.rotulo.apply(adapto_faixa)


In [18]:
novo_df.tail()

,resumo,rotulo
163,Rio de Janeiro (RJ) – O Centro de Avaliações d...,2
164,Este trabalho apresenta um sistema para contro...,2
165,No contexto das comunicações táticas baseadas ...,2
166,O valor da velocidade de alvos móveis em image...,0
167,Neste artigo é apresentada a análise da seção ...,0


In [19]:
class_names = ['Faixa 1', 'Faixa 2', 'Faixa 3']

### gpt2-small-portuguese

- https://huggingface.co/pierreguillou/gpt2-small-portuguese

```
@inproceedings{pierre2020gpt2smallportuguese,
  title={GPorTuguese-2 (Portuguese GPT-2 small): a Language Model for Portuguese text generation (and more NLP tasks...)},
  author={Pierre Guillou},
  year={2020}
}

```


- **Tokenizer**: No GPT-2, ao contrário do BERT, o preenchimento é feito à esquerda, uma vez que o último token é utilizado para a previsão.

In [20]:
PRE_TRAINED_MODEL_NAME = "pierreguillou/gpt2-small-portuguese"
# tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer = GPT2Tokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
# tokenizer.model_max_length=MAX_LEN
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [21]:
model = GPT2Model.from_pretrained(PRE_TRAINED_MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [22]:
frase = 'A avaliação de prontidão tecnológica em tecnologias de interesse militar'

In [23]:
gpt2_input = tokenizer(frase, padding="max_length", max_length=16, truncation=True, return_tensors="pt")

In [24]:
print(gpt2_input['input_ids'])
print(gpt2_input["attention_mask"])

tensor([[    0,     0,     0,     0,     0,    33,  7912,   261,   374, 38198,
         20142,   300,  9643,   261,  3325,  2303]])
tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [25]:
example_text = tokenizer.decode(gpt2_input.input_ids[0])
print(example_text)

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>A avaliação de prontidão tecnológica em tecnologias de interesse militar


In [26]:
encoding = tokenizer.encode_plus(
  frase,
  max_length=20,   #Perceba que irei cortar um token da minha frase anterior
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


dict_keys(['input_ids', 'attention_mask'])

- Informações sobre o modelo:

In [27]:
model = model.to(device)

In [28]:
model.config.hidden_size

768

In [29]:
model.config

GPT2Config {
  "_name_or_path": "pierreguillou/gpt2-small-portuguese",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [30]:
# https://medium.com/@TaaniyaArora/visualizing-gpt2-word-embeddings-on-tensorboard-ea5c8fef9efa
# gpt2_word_embeddings = model.wte.weight # Word Token Embeddings
# print(gpt2_word_embeddings.shape)
# gpt2_word_embeddings
# gpt2_word_embeddings = model.transformer.wte.weight # Word Token Embeddings
# print(gpt2_word_embeddings.shape)

# Aplicando o Modelo para gerar a representação vetorial


- Dúvidas e explicações sobre o GPT-2 para Feature Extraction
```
1 - https://stackoverflow.com/questions/60574112/can-we-use-gpt-2-sentence-embedding-for-classification-tasks
```
```
2 - https://github.com/huggingface/transformers/issues/3168
```



- Não encontrei um tutorial que fosse propriamente para o GPT-2, mas foi possível adaptar os tutoriais de BERT para ele, devido as similaridades das arquiteturas baseadas em *transformers*.

- Tutorial interessante:

```
1 - https://towardsdatascience.com/feature-extraction-with-bert-for-text-classification-533dde44dc2f
```

```
2 - https://github.com/felipemaiapolo/legalnlp/blob/main/demo/BERT/BERT_TUTORIAL.ipynb
```

*Recomendo muito o 2, pois foi feito em formato de notebook, bem redigido, comentando os passos e explicando o BERT. ALém de estar em português (material raro em termos de qualidade em nossa língua)*

- Um cuidado que se deve ter é em relação a máscara de atenção, em ambos tutoriais, ela não era passada, o que gerava um ALERTA. Consultando a docuemntação, vi como deve ser passado, e ao final, os modelos tiveram um ganho de desempenho.

### Transformando os dados em tensores:

In [31]:
# Fazendo a padronização dos textos
wrapper = textwrap.TextWrapper()
data_resumo = list(novo_df['resumo'])

- Visualizando:

In [32]:
for text in range(len(data_resumo[:2])):
  print(f'{wrapper.fill(data_resumo[text])}')
  print()

O crescente emprego de mísseis de ombro infravermelhos contra alvos
aéreos demanda a utilização de contramedidas cada vez mais modernas e
eficientes. Neste cenário, surge o Directed Infrared Countermeasure
(DIRCM), cujo objetivo é interferir no guiamento do míssil por meio de
pulsos de laser. Neste artigo, um seeker infravermelho do tipo rising
sun é modelado e simulado, sendo os efeitos da emissão de um DIRCM no
processamento do sinal avaliados. A influência de parâmetros de
frequência de repetição de pulsos e intensidade do laser são
evidenciados. Os resultados obtidos ressaltam a importância e a
necessidade do desenvolvimento de ferramentas computacionais mais
complexas, visando ao desenvolvimento da doutrina de emprego deste
tipo de contramedida.

Materiais dielétricos com baixas perdas e alta permissividade são
componentes essenciais que são utilizados em linhas de transmissões
não lineares capacitivas (LTNLs) na geração de RF. LTNLs possuem
grande potencial para gerar ondas de só

In [33]:
# Aplicando o bert_tokenizer com o comprimento máximo que definimos
encoded_inputs = tokenizer(data_resumo, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")


#O encoded_input está como um dicionário com 3 chaves
encoded_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

- Visualizando como o resumo ficou tokenizado, i.e., a chave ''input_ids' do encoded_inputs :

In [34]:
#Visualiando o primeiro texto após a aplicação do tokenizador
print(encoded_inputs['input_ids'][0])

print()

# Mostrando o mesmo texto decodificado
print(wrapper.fill(tokenizer.decode(encoded_inputs['input_ids'][0])))

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    47,  6660,  7124,   261, 16013,   261,
        20825,  6063,   512, 39959,   719, 15396, 19255,  8671,   259,  5663,
          261,   719,  8846,   777,  1181,   699,   447, 11371,   258, 24086,
           14,  4651,  5529,    12,  8469,   275, 34541,  3722, 

In [35]:
print(encoded_inputs['input_ids'].shape)  # batch size x seq length
print(encoded_inputs['attention_mask'].shape)

torch.Size([168, 242])
torch.Size([168, 242])


In [36]:
# Passando os tensores para para a GPU - aceleração através da GPU
input_ids = encoded_inputs['input_ids'].to(device)
attention_mask = encoded_inputs['attention_mask'].to(device)
# encoded_inputs = encoded_inputs.to(device)


# input_ids = encoded_inputs['input_ids']
# attention_mask = encoded_inputs['attention_mask']

In [37]:
# Criando o vetor de features/caracteristicas
features = []

- buscando entender sobre a classe model e o seu retorno

In [38]:
type(model(input_ids[0:(1)]))

transformers.modeling_outputs.BaseModelOutputWithPastAndCrossAttentions

In [39]:
type(model(input_ids[0:(1)])[1])

tuple

In [40]:
len(model(input_ids[0:(1)])[1])

12

In [41]:
model(input_ids[0:(1)]).last_hidden_state

tensor([[[-1.9671e-01,  1.7107e-01,  1.0749e-01,  ...,  3.2468e-03,
           1.4821e-01, -2.3598e-01],
         [-2.9810e-01, -5.4413e-03, -2.9641e-01,  ...,  8.9274e-01,
          -1.7007e-01, -6.9236e-01],
         [-3.9560e-01,  2.4554e-01, -3.1330e-01,  ...,  7.9841e-01,
           1.1377e-01,  1.1075e-01],
         ...,
         [ 3.3436e+00,  2.8500e+00, -1.3963e+00,  ...,  3.1525e-01,
           2.4417e-01,  2.2235e-01],
         [ 8.4440e-01,  1.1935e-01, -1.0324e+00,  ...,  8.2011e-01,
           4.7389e-01,  1.9297e+00],
         [ 3.4758e-01, -8.4889e-01, -1.0813e+00,  ...,  3.5444e-01,
           2.8225e-01,  8.6339e-01]]], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [42]:
model(input_ids[0:(1)]).last_hidden_state.cpu

<function Tensor.cpu>

In [43]:
# model(**encoded_inputs[0:(1)]).last_hidden_state

- Aplicando o GPT-2 para a extração das características

Documentação oficial que precisei estudar, para ocnseguir adaptar o processo do BERT para o GPT-2:

```
1 - https://huggingface.co/transformers/v4.9.2/model_doc/gpt2.html#gpt2model
```



In [44]:
# for i in tqdm(range(len(data_resumo))):
#     with torch.no_grad():
#       last_hidden_states = model(input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
#     features.append(last_hidden_states)

for i in tqdm(range(len(data_resumo))):
    with torch.no_grad():
      # last_hidden_states = model(input_ids[i:(i+1)],attention_mask[i:(i+1)]).last_hidden_state.cpu().numpy().reshape(-1).tolist()
      last_hidden_states = model(input_ids[i:(i+1)]).last_hidden_state.cpu().numpy().reshape(-1).tolist()
      # last_hidden_states = model(**encoded_inputs).last_hidden_state.cpu().numpy().reshape(-1).tolist()
    features.append(last_hidden_states)

100%|██████████| 168/168 [00:04<00:00, 39.80it/s]


In [45]:
# passando a lista features para numpy array com as features extraidas
features = np.array(features)

- Visualizando o resultado:

In [46]:
print('FEATURES: Número de linhas: ' + str(features.shape[0]) + ' Número de colunas: ', str(features.shape[1]))
print("FEATURES é um objeto " + str(type(features)))

FEATURES: Número de linhas: 168 Número de colunas:  185856
FEATURES é um objeto <class 'numpy.ndarray'>


In [47]:
features[0]

array([-0.19670703,  0.17107086,  0.1074942 , ...,  0.35444483,
        0.28225058,  0.86338526])

# Adaptando a representação vetorial gerada

- Perceba que até aqui, temos um numpy array chamado **features**, com 168 linhas e 768 colunas.

- Iremos transformar nosso numpy array em um dataframe novamente:

In [48]:
df_repvet = pd.DataFrame(features)
repvet_rotulado = pd.concat([df_repvet, novo_df['rotulo']], axis = 1)
repvet_rotulado.shape

(168, 185857)

In [49]:
df_repvet

,0,1,2,3,4,5,6,7,8,9,...,185846,185847,185848,185849,185850,185851,185852,185853,185854,185855
0,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.333590,-0.330011,0.461910,-0.131250,-0.144836,0.343001,0.317489,0.354445,0.282251,0.863385
1,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.063566,-0.424902,0.141903,-0.706703,-0.705978,0.436493,0.363464,0.535535,0.395689,1.161974
2,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,-0.007008,-1.298491,0.474298,-0.669279,-0.514898,-0.210728,0.064951,0.259435,0.572413,0.527668
3,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.213110,-0.820673,-0.200807,-1.008460,-0.209426,-0.027674,0.580211,0.458663,0.435319,0.215276
4,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.373481,-0.918610,0.301205,-0.083270,0.075774,0.856035,0.586876,0.459331,0.173122,0.873856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.047160,-0.133200,0.218046,0.004687,-0.11342,-0.396993,0.068496,-0.319661,-0.250460,0.113693,...,0.022715,1.039169,0.199684,-0.142284,-0.029440,-0.934128,0.494391,0.339530,-0.015037,0.745815
164,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.557624,0.862611,0.113057,0.851308,-1.437908,-0.703424,-0.889538,1.697932,-0.462616,0.551399
165,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.468734,-0.104452,0.696173,-0.671978,-0.214931,0.952818,0.575520,0.273585,0.000443,0.675025
166,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.136430,-0.465639,0.191863,-0.575489,-0.578803,-0.166991,0.610428,0.528582,0.098336,0.364607


In [50]:
repvet_rotulado.head(3)

,0,1,2,3,4,5,6,7,8,9,...,185847,185848,185849,185850,185851,185852,185853,185854,185855,rotulo
0,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,-0.330011,0.461910,-0.131250,-0.144836,0.343001,0.317489,0.354445,0.282251,0.863385,0
1,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,-0.424902,0.141903,-0.706703,-0.705978,0.436493,0.363464,0.535535,0.395689,1.161974,1
2,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,-1.298491,0.474298,-0.669279,-0.514898,-0.210728,0.064951,0.259435,0.572413,0.527668,0


# Criando uma estrutura para armazenar os resultados

In [51]:
# Algoritmos de classificação
classifiers = [  #NB, KNN, SVM
    ('Multinomial Naive Bayes', MultinomialNB()),
    ('Complement Naive Bayes Classifier', ComplementNB()),
    ('KNN', KNeighborsClassifier()),  #n_neighbors default é 5
    ('SVM', svm.SVC( )),
     ('Random Forest', RandomForestClassifier(random_state=42)),
      ('AdaBoost',    AdaBoostClassifier(random_state=42)) #n_estimators default é 50
]

In [52]:
lista_classificador_nome = list()
for classifier_name, classifier in classifiers:
    lista_classificador_nome.append(classifier_name)

In [53]:
df_acc = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])
df_f1 = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])
df_f1_ponderado = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])

In [54]:
for classifier_name, classifier in classifiers:
    nova_linha = pd.DataFrame({'Classificador': [classifier_name], 'Rodada 1':[0] , 'Rodada 2':[0], 'Rodada 3':[0], 'Rodada 4':[0], 'Rodada 5':[0], 'Media':[0]})
    df_acc = pd.concat([df_acc, nova_linha], ignore_index=True)
    df_f1 = pd.concat([df_f1, nova_linha], ignore_index=True)
    df_f1_ponderado = pd.concat([df_f1_ponderado, nova_linha], ignore_index=True)

In [55]:
df_f1

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


In [56]:
df_f1_ponderado

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


In [57]:
df_acc

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


 Usei como parâmetro para **average** o **'macro'** para o f1, e o **'weighted'** para o f1-ponderado

**'weighted'**:

Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

**'macro'**:

Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

 vide [Documentação oficial](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)

# Aplicando modelo de classificação

In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support as score

def evaluate_model(model, X_test, y_test):
    # Predição dos rótulos
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

    # Cálculo da matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    # Cálculo da acurácia
    acc = accuracy_score(y_test, y_pred)

    # Cálculo do F1-score
    f1 = f1_score(y_test, y_pred, average='macro')

    # Cálculo do F1-score
    f1_poderado = f1_score(y_test, y_pred, average='weighted')

    # Outras métricas
    precision, recall, f1score, support = score(y_test, y_pred, average='macro')
    return cm, acc, f1, precision, recall, f1score, support, f1_poderado

In [59]:
df_repvet

,0,1,2,3,4,5,6,7,8,9,...,185846,185847,185848,185849,185850,185851,185852,185853,185854,185855
0,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.333590,-0.330011,0.461910,-0.131250,-0.144836,0.343001,0.317489,0.354445,0.282251,0.863385
1,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.063566,-0.424902,0.141903,-0.706703,-0.705978,0.436493,0.363464,0.535535,0.395689,1.161974
2,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,-0.007008,-1.298491,0.474298,-0.669279,-0.514898,-0.210728,0.064951,0.259435,0.572413,0.527668
3,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.213110,-0.820673,-0.200807,-1.008460,-0.209426,-0.027674,0.580211,0.458663,0.435319,0.215276
4,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.373481,-0.918610,0.301205,-0.083270,0.075774,0.856035,0.586876,0.459331,0.173122,0.873856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.047160,-0.133200,0.218046,0.004687,-0.11342,-0.396993,0.068496,-0.319661,-0.250460,0.113693,...,0.022715,1.039169,0.199684,-0.142284,-0.029440,-0.934128,0.494391,0.339530,-0.015037,0.745815
164,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.557624,0.862611,0.113057,0.851308,-1.437908,-0.703424,-0.889538,1.697932,-0.462616,0.551399
165,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.468734,-0.104452,0.696173,-0.671978,-0.214931,0.952818,0.575520,0.273585,0.000443,0.675025
166,-0.196707,0.171071,0.107494,0.125283,-0.16889,-0.319846,0.274585,-0.134318,-0.265486,-0.019445,...,0.136430,-0.465639,0.191863,-0.575489,-0.578803,-0.166991,0.610428,0.528582,0.098336,0.364607


- Alguns algoritmos só lidam com valores positivos, por isso precisamos normalizar os valores que aparecem na RV do BERT e GPT-2:

In [60]:
# scaler = MinMaxScaler()
# df_repvet_pos = scaler.fit_transform(df_repvet)
# scaling_factor = 1000
# df_repvet_pos_i = (df_repvet_pos * scaling_factor).astype(int)
# df_repvet_pos_i = pd.DataFrame(df_repvet_pos_i, columns=df_repvet.columns)

In [61]:
# df_repvet_pos_i.describe()

In [62]:
classifiers

[('Multinomial Naive Bayes', MultinomialNB()),
 ('Complement Naive Bayes Classifier', ComplementNB()),
 ('KNN', KNeighborsClassifier()),
 ('SVM', SVC()),
 ('Random Forest', RandomForestClassifier(random_state=42)),
 ('AdaBoost', AdaBoostClassifier(random_state=42))]

In [63]:
classificador=0
for classifier_name, classifier in classifiers:
    print('---', classifier_name, '---')
    y_true = []
    y_pred = []
    contador = 0
    serie_acc = pd.Series()
    serie_f1 = pd.Series()
    serie_f1_ponderado = pd.Series()


    for i in range(0, 5):  #Estou percorrendo as 5 rodadas
        train_index = kfold_train[i]
        test_index = kfold_test[i]
        contador +=1

         # df_repvet
        X_train_n, X_test_n = df_repvet.iloc[train_index], df_repvet.iloc[test_index]
        y_train, y_test = repvet_rotulado.iloc[train_index]['rotulo'], repvet_rotulado.iloc[test_index]['rotulo']


        scaler = MinMaxScaler()
        df_repvet_pos_treino = scaler.fit_transform(X_train_n)
        df_repvet_pos_teste =  scaler.transform(X_test_n)

        scaling_factor = 1000
        df_repvet_pos_i_treino = (df_repvet_pos_treino * scaling_factor).astype(int)
        X_train = pd.DataFrame(df_repvet_pos_i_treino, columns=df_repvet.columns)

        df_repvet_pos_i_teste = (df_repvet_pos_teste * scaling_factor).astype(int)
        X_test = pd.DataFrame(df_repvet_pos_i_teste, columns=df_repvet.columns)

        # Treinamento do modelo
        classifier.fit(X_train, y_train)
        cm, acc, f1, precision, recall, f1score, support,f1_poderado = evaluate_model(classifier, X_test, y_test)


        print(classifier_name + " Rodada " + str(contador) )
        print('Matriz de Confusão:')
        print(cm)
        print('Acurácia:', acc)
        print('F1-Score:', f1)
        print("outras métricas:")
        print('precision:', precision)
        print('recall:', recall)
        print('f1score:', f1score)
        print('support:', support)
        print('-------------------------------------')
        # serie_acc = serie_acc.append(pd.Series([acc]))
        serie_acc = pd.concat([serie_acc, pd.Series([acc])])
        # serie_f1 = serie_f1.append(pd.Series([f1]))
        serie_f1 = pd.concat([serie_f1, pd.Series([f1])])
        serie_f1_ponderado = pd.concat([serie_f1_ponderado, pd.Series([f1_poderado])])


    # Avaliação do modelo: Aqui estamos inserindo os valores das medias na serie
    media_acc = serie_acc[:5].mean()
    media_f1 = serie_f1[:5].mean()
    media_f1_ponderado = serie_f1_ponderado[:5].mean()
    # serie_acc = serie_acc.append(pd.Series([media_acc]))
    # serie_f1 = serie_f1.append(pd.Series([media_f1]))
    serie_acc = pd.concat([serie_acc, pd.Series([media_acc])])
    serie_f1 = pd.concat([serie_f1, pd.Series([media_f1])])
    serie_f1_ponderado = pd.concat([serie_f1_ponderado, pd.Series([media_f1_ponderado])])

    # print("Acurácia: " )
    # print(serie_acc)
    # print("F-1: " )
    # print(serie_f1)
    df_acc.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_acc.values
    df_f1.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_f1.values
    df_f1_ponderado.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_f1_ponderado.values
    classificador+=1
    print("=======================================================================================")
    # cm = confusion_matrix(y_true, y_pred)
    # acc = accuracy_score(y_true, y_pred)

--- Multinomial Naive Bayes ---


<ipython-input-63-d6486795cf5a>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-63-d6486795cf5a>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-63-d6486795cf5a>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()


              precision    recall  f1-score   support

           0       0.81      0.68      0.74        19
           1       0.29      0.25      0.27         8
           2       0.64      1.00      0.78         7

    accuracy                           0.65        34
   macro avg       0.58      0.64      0.60        34
weighted avg       0.65      0.65      0.64        34

Multinomial Naive Bayes Rodada 1
Matriz de Confusão:
[[13  5  1]
 [ 3  2  3]
 [ 0  0  7]]
Acurácia: 0.6470588235294118
F1-Score: 0.5957671957671958
outras métricas:
precision: 0.5781926406926406
recall: 0.6447368421052632
f1score: 0.5957671957671958
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.65      0.79      0.71        19
           1       0.25      0.12      0.17         8
           2       0.71      0.71      0.71         7

    accuracy                           0.62        34
   macro avg       0.54      0.54      0.53  

<ipython-input-63-d6486795cf5a>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-63-d6486795cf5a>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-63-d6486795cf5a>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib

              precision    recall  f1-score   support

           0       0.76      0.84      0.80        19
           1       0.00      0.00      0.00         8
           2       0.54      1.00      0.70         7

    accuracy                           0.68        34
   macro avg       0.43      0.61      0.50        34
weighted avg       0.54      0.68      0.59        34

Complement Naive Bayes Classifier Rodada 1
Matriz de Confusão:
[[16  0  3]
 [ 5  0  3]
 [ 0  0  7]]
Acurácia: 0.6764705882352942
F1-Score: 0.5
outras métricas:
precision: 0.4334554334554334
recall: 0.6140350877192983
f1score: 0.5
support: None
-------------------------------------


...

              precision    recall  f1-score   support

           0       0.67      1.00      0.80        18
           1       0.00      0.00      0.00         9
           2       0.83      0.83      0.83         6

    accuracy                           0.70        33
   macro avg       0.50      0.61      0.54        33
weighted avg       0.52      0.70      0.59        33

SVM Rodada 5
Matriz de Confusão:
[[18  0  0]
 [ 8  0  1]
 [ 1  0  5]]
Acurácia: 0.696969696969697
F1-Score: 0.5444444444444444
outras métricas:
precision: 0.5
recall: 0.6111111111111112
f1score: 0.5444444444444444
support: None
-------------------------------------
--- Random Forest ---
              precision    recall  f1-score   support

           0       0.83      1.00      0.90        19
           1       0.50      0.12      0.20         8
           2       0.67      0.86      0.75         7

    accuracy                           0.76        34
   macro avg       0.66      0.66      0.62        34
weigh

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.70      1.00      0.83        19
           1       0.00      0.00      0.00         8
           2       0.67      0.67      0.67         6

    accuracy                           0.70        33
   macro avg       0.46      0.56      0.50        33
weighted avg       0.53      0.70      0.60        33

Random Forest Rodada 4
Matriz de Confusão:
[[19  0  0]
 [ 6  0  2]
 [ 2  0  4]]
Acurácia: 0.696969696969697
F1-Score: 0.4975845410628019
outras métricas:
precision: 0.4567901234567901
recall: 0.5555555555555555
f1score: 0.4975845410628019
support: None
-------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.64      0.89      0.74        18
           1       0.00      0.00      0.00         9
           2       0.50      0.67      0.57         6

    accuracy                           0.61        33
   macro avg       0.38      0.52      0.44        33
weighted avg       0.44      0.61      0.51        33

Random Forest Rodada 5
Matriz de Confusão:
[[16  0  2]
 [ 7  0  2]
 [ 2  0  4]]
Acurácia: 0.6060606060606061
F1-Score: 0.4385382059800665
outras métricas:
precision: 0.38000000000000006
recall: 0.5185185185185185
f1score: 0.4385382059800665
support: None
-------------------------------------
--- AdaBoost ---
              precision    recall  f1-score   support

           0       0.55      0.84      0.67        19
           1       0.00      0.00      0.00         8
           2       1.00      0.14      0.25         7

    accuracy                           0.50        34
   macro avg       0.52      0.33    

In [64]:
df_acc

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.647059,0.617647,0.647059,0.606061,0.606061,0.624777
1,Complement Naive Bayes Classifier,0.676471,0.617647,0.647059,0.69697,0.606061,0.648841
2,KNN,0.705882,0.588235,0.617647,0.636364,0.606061,0.630838
3,SVM,0.764706,0.705882,0.735294,0.69697,0.69697,0.719964
4,Random Forest,0.764706,0.647059,0.705882,0.69697,0.606061,0.684135
5,AdaBoost,0.5,0.558824,0.558824,0.515152,0.515152,0.52959


In [65]:
df_f1

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.595767,0.531746,0.601221,0.550427,0.519118,0.559656
1,Complement Naive Bayes Classifier,0.5,0.459259,0.47619,0.48661,0.452236,0.474859
2,KNN,0.550933,0.446377,0.54025,0.479798,0.536869,0.510845
3,SVM,0.569083,0.541667,0.577196,0.497585,0.544444,0.545995
4,Random Forest,0.618254,0.477541,0.552381,0.497585,0.438538,0.51686
5,AdaBoost,0.305556,0.394525,0.467387,0.34872,0.427642,0.388766


In [66]:
df_f1_ponderado

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.638002,0.585434,0.633139,0.599223,0.565084,0.604176
1,Complement Naive Bayes Classifier,0.591176,0.534641,0.563025,0.598083,0.512749,0.559935
2,KNN,0.632965,0.536573,0.579922,0.566116,0.577686,0.578652
3,SVM,0.663394,0.613971,0.64186,0.596838,0.587879,0.620788
4,Random Forest,0.707073,0.56529,0.623529,0.596838,0.509816,0.600509
5,AdaBoost,0.42402,0.504547,0.550606,0.476071,0.481596,0.487368
